![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=SocialStudies/OrphanWells/orphan-wells.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Orphan Wells in Alberta

In Alberta there are a number of oil and gas well sites that are no longer producing oil or gas. Well sites that don't have a company responsible for cleaning them up are called [orphan wells](https://en.wikipedia.org/wiki/Orphan_wells_in_Alberta,_Canada).

## Orphan Well Association

The [Orphan Well Association](https://www.orphanwell.ca) has been established to "close wells, facilities and pipelines that do not have a solvent and responsible owner—known as orphans—to protect people and the environment, and remove the potential risk of unfunded liability." Using data from [orphanwell.ca](https://www.orphanwell.ca/about/orphan-inventory) as of 2021-12-14, we can visualize where these orphan wells are in Alberta.

Select the following code cell, then click the `▶Run` button to load the data into a [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/SocialStudies/OrphanWells/orphan-wells-alberta.csv')
df

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)>

There are 8683 rows, which means that 8683 orphan wells have been identified in Alberta.

Of course there is a diclaimer about this data set:

> *The Orphan Well Association or OWA makes no representation, warranties, or guarantees, expressed or implied, for the fitness of the data with respect to its use. The OWA is not responsible for any costs incurred from use of the information in this list.*
*The AER Licence Details Report includes multiple events for individual wells in order to report production. Therefore there will be more entries than number of wells. Users of this list are advised that it does not necessarily reflect the current inventory of orphan wells held by the OWA and users are responsible for confirming any information used.*

We can check how many wells there are in the data set with each status. “Closed” means it has been decommissioned, while “Active” means decommissioning is pending or it needs further downhole work.

In [ ]:
for status in df['Status'].unique():
    print(status, df[df['Status'] == status].shape[0])

### Map

Let's generate an interactive map showing the locations of these sites. Sites designated `Closed` will have purple markers, and `Active` will be red.

You can zoom in and out on the map. Clicking on a marker cluster will zoom in and expand it. Clicking on a marker will show you its [Unique Well Identifier](http://www.310sign.ca/blog/what-is-a-uwi-unique-well-identifier.html) and status.

In [2]:
import folium
from folium.plugins import MarkerCluster
m = folium.Map(location=[53.5,-114], zoom_start=5)
mc = MarkerCluster()
for i, row in df.iterrows():
    if row['Status'] == 'Closed':
        color = 'purple'
    if row['Status'] == 'Active':
        color = 'red'
    mc.add_child(folium.Marker(location=[row['Surface Latitude'], row['Surface Longitude']],
                               popup=row['Unique Well Identifier'] + ' ' + row['Status'],
                               icon=folium.Icon(color=color)))
m.add_child(mc)
m

NameError: name 'df' is not defined

## WellWiki

Another website, [WellWiki](https://www.wellwiki.org), maintains information about oil and gas wells in Canada and the United States. There are currently [619 503 Alberta wells](https://www.wellwiki.org/wiki/Special:AlbertaAdvancedWellSearch) on the site.

Let's load data about the wells that have a license status of "Abandoned". This will take about a minute to load, on the left of the cell you'll see `In [*]` which means that it is running.

In [ ]:
import warnings, requests
url = 'https://www.wellwiki.org/wiki/Special:AlbertaAdvancedWellSearch?operatorfiltertype=matches&operatorfiltervalue=&countyfiltertype=matches&countyfiltervalue=&datefiltertype=matches&datefiltervalue=&licensefiltertype=matches&licensefiltervalue=Abandoned&meridianfiltertype=matches&meridianfiltervalue=&townshipfiltertype=matches&townshipfiltervalue=&rangefiltertype=matches&rangefiltervalue=&sectionfiltertype=matches&sectionfiltervalue=&pagenum=&pagesize=100000'

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    res = requests.get(url, verify=False)
ww = pd.read_html(res.text)[1]
ww.columns = ['UWI','Operator','County/Municipality','Meridian','Township','Range','Section','License Date','License Status']
ww

This data set shows 93 642 wells that have a license status of "Abandoned". If we want more information about an individual well, we can go to its page on the WellWiki site using the UWI (e.g. for well 0054022715000 it is https://www.wellwiki.org/wiki/0054022715000).

Unfortunately the `UWI` column doesn't follow the same format as the `Unique Well Identifier` from the previous data set. However `UWI` does contain the information we need: sequence, location (Township, Meridian, Range, Section, and Legal Subdivision), and event.

We'll use this information to create a new column in the `ww` DataFrame called `Unique Well Identifier`.

In [ ]:
def uwi_translator(UWI):
    TWP = UWI[:3]
    Meridian = UWI[3]
    Range = UWI[4:6]
    Section = UWI[6:8]
    LSD = UWI[8:10]
    Sequence = UWI[11]
    Event = UWI[12]
    new_uwi = '10'+Sequence+'/'+LSD+'-'+Section+'-'+TWP+'-'+Range+'W'+Meridian+'/'+Event
    return new_uwi

well_ids = []
for row in ww.itertuples():
    well_ids.append(uwi_translator(row.UWI))
ww['Unique Well Identifier'] = well_ids
ww

### Unique Locations

We can also create a new column called `Location Code` and create a new data set that includes only the first well with each location code.

In [ ]:
locations = []
for row in ww.itertuples():
    locations.append(row.UWI[2:11])
ww['Location Code'] = locations
ww_unique = ww.drop_duplicates(subset='Location Code', keep='first')
ww_unique

Next we could load the WellWiki page for each of those `UWI` values and extract the latitude and longitude of the well using the following code:

```python
latitudes = []
longitudes = []
for row in ww_unique.itertuples():
    site = pd.read_html('https://www.wellwiki.org/wiki/'+row.UWI)
    latitudes.append(site[0][0][19].split(': ')[1])
    longitudes.append(site[0][0][20].split(': ')[1])
ww_unique['Latitude'] = latitudes
ww_unique['Longitude'] = longitudes
ww_unique
```

Unfortunately this would take quite a while, and the website might not appreciate all those page loads. Instead we will read the locations from a CSV file we have prepared, and then generate a map of these abandoned well locations.

Since there are a lot of locations this will take a minute or two to generate and render. 

In [ ]:
try:
    wwu = pd.read_csv('wellwiki-abandoned-unique.csv')
except:
    wwu = pd.read_csv('https://raw.githubusercontent.com/callysto/data-files/main/SocialStudies/OrphanWells/wellwiki-abandoned-unique.csv')
wwu = wwu.set_index('Unnamed: 0')
wwu.index.name = None
print('Generating a map of', wwu.shape[0], 'abandoned well locations.')
m2 = folium.Map(location=[53.5,-114], zoom_start=5)
mc2 = MarkerCluster()
for row in wwu.itertuples():
    mc2.add_child(folium.Marker(location=[row.Latitude,row.Longitude], popup='https://www.wellwiki.org/wiki/'+row.UWI))
m2.add_child(mc2)
m2

In [3]:
def uwi_translator(UWI):
    TWP = UWI[:3]
    Meridian = UWI[3]
    Range = UWI[4:6]
    Section = UWI[6:8]
    LSD = UWI[8:10]
    Sequence = UWI[11]
    Event = UWI[12]
    new_uwi = '10'+Sequence+'/'+LSD+'-'+Section+'-'+TWP+'-'+Range+'W'+Meridian+'/'+Event
    return new_uwi


aer = pd.read_csv('/Users/ayushchaudhari/Downloads/AppData/VAR/VAR3500Report/TXT/2023/AERWellList.csv',dtype='unicode')

aer['Unique Well Identifier'] = aer['UWI'].map(lambda x: uwi_translator(x))

aer

,WELL ID,UWI,WELL NAME,FIELD CODE,POOL CODE,OS-AREA CODE,OS-DEP- CODE,LIC NO,LIC STATUS,LIC-ISSUE-DATE,...,WELL TD,WELL-STAT-CODE,WELL-STAT-DATE,FLUID,MODE,PURPOSE,STURCTURE,LONGITUDE,LATITUDE,Unique Well Identifier
0,00/08-09-051-25W4/0,0514250908000,"113198REDWATER NO. 2,",0551,0000000,,0000000,0000001,RecExempt,19490523,...,1317.30,0002000000,19490705,N/A,ABD,N/A,N/A,-113.617905,53.386619,100/08-09-051-25W4/0
1,02/05-25-049-26W4/0,0494262505020,LEDUC CALMAR #1A,0551,0000000,,0000000,0000002,RecExempt,19490527,...,1616.40,0002000000,19490812,N/A,ABD,N/A,N/A,-113.680744,53.256289,102/05-25-049-26W4/0
2,00/07-11-062-23W4/0,0624231107000,STANOLIND-ROCHESTER NO. 1 WELL,0003,0000000,,0000000,0000003,RecExempt,19490527,...,1362.20,0002000000,19490721,N/A,ABD,N/A,N/A,-113.334918,54.346682,100/07-11-062-23W4/0
3,00/11-04-051-26W4/2,0514260411002,IMPERIAL WOODBEND NO. 20,0551,0244412,,0000000,0000006,Abandoned,19490528,...,1618.50,0102000000,20220307,CR-OIL,ABD,N/A,N/A,-113.778326,53.376122,100/11-04-051-26W4/2
4,00/07-21-057-21W4/0,0574212107000,BREVIA007 REDWATER 7-21-57-21,0770,0720000,,0000000,0000007,Suspension,19490508,...,0979.30,0101000000,20170201,CR-OIL,SUSP,N/A,N/A,-113.059581,53.939876,100/07-21-057-21W4/0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504578,AA/15-20-095-10W4/0,0954102015AA0,SHELL MUSKEG OV A2,0998,0000000,A001,0000000,Y0003293,RecExempt,19600520,...,0082.60,0002000000,19600521,N/A,ABD,N/A,N/A,-111.589776,57.262457,10A/15-20-095-10W4/0
504579,AA/13-08-098-10W4/0,0984100813AA0,OILSANDS EVALUATION 120,0998,0000000,A001,0000000,Y0003294,RecExempt,19570301,...,0050.00,0002000000,19570302,N/A,ABD,N/A,N/A,-111.598204,57.493724,10A/13-08-098-10W4/0
504580,AA/15-08-098-10W4/0,0984100815AA0,OILSANDS EVALUATION 128,0998,0000000,A001,0000000,Y0003295,RecExempt,19570301,...,0067.00,0002000000,19570302,N/A,ABD,N/A,N/A,-111.590576,57.493725,10A/15-08-098-10W4/0
504581,BZ/07-27-085-08W4/0,0854082707BZ0,AMOCO FO-40 GREGOIRE EX AD7-27-85-8,0998,0000000,A001,0000000,Z0000001,RecCertified,19740206,...,0342.90,0002000000,19760522,N/A,ABD,N/A,N/A,-111.17259,56.396349,10Z/07-27-085-08W4/0


In [10]:
aer.loc[aer['MODE'].str.strip() == 'ABD']


,WELL ID,UWI,WELL NAME,FIELD CODE,POOL CODE,OS-AREA CODE,OS-DEP- CODE,LIC NO,LIC STATUS,LIC-ISSUE-DATE,...,WELL TD,WELL-STAT-CODE,WELL-STAT-DATE,FLUID,MODE,PURPOSE,STURCTURE,LONGITUDE,LATITUDE,Unique Well Identifier
0,00/08-09-051-25W4/0,0514250908000,"113198REDWATER NO. 2,",0551,0000000,,0000000,0000001,RecExempt,19490523,...,1317.30,0002000000,19490705,N/A,ABD,N/A,N/A,-113.617905,53.386619,100/08-09-051-25W4/0
1,02/05-25-049-26W4/0,0494262505020,LEDUC CALMAR #1A,0551,0000000,,0000000,0000002,RecExempt,19490527,...,1616.40,0002000000,19490812,N/A,ABD,N/A,N/A,-113.680744,53.256289,102/05-25-049-26W4/0
2,00/07-11-062-23W4/0,0624231107000,STANOLIND-ROCHESTER NO. 1 WELL,0003,0000000,,0000000,0000003,RecExempt,19490527,...,1362.20,0002000000,19490721,N/A,ABD,N/A,N/A,-113.334918,54.346682,100/07-11-062-23W4/0
3,00/11-04-051-26W4/2,0514260411002,IMPERIAL WOODBEND NO. 20,0551,0244412,,0000000,0000006,Abandoned,19490528,...,1618.50,0102000000,20220307,CR-OIL,ABD,N/A,N/A,-113.778326,53.376122,100/11-04-051-26W4/2
5,00/09-13-052-01W5/0,0525011309000,OPPORTUNITY NO. 1 WELL,0998,0000000,,0000000,0000008,RecExempt,19490527,...,1830.00,0102000000,19490721,CR-OIL,ABD,N/A,N/A,-114.005028,53.492088,100/09-13-052-01W5/0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504578,AA/15-20-095-10W4/0,0954102015AA0,SHELL MUSKEG OV A2,0998,0000000,A001,0000000,Y0003293,RecExempt,19600520,...,0082.60,0002000000,19600521,N/A,ABD,N/A,N/A,-111.589776,57.262457,10A/15-20-095-10W4/0
504579,AA/13-08-098-10W4/0,0984100813AA0,OILSANDS EVALUATION 120,0998,0000000,A001,0000000,Y0003294,RecExempt,19570301,...,0050.00,0002000000,19570302,N/A,ABD,N/A,N/A,-111.598204,57.493724,10A/13-08-098-10W4/0
504580,AA/15-08-098-10W4/0,0984100815AA0,OILSANDS EVALUATION 128,0998,0000000,A001,0000000,Y0003295,RecExempt,19570301,...,0067.00,0002000000,19570302,N/A,ABD,N/A,N/A,-111.590576,57.493725,10A/15-08-098-10W4/0
504581,BZ/07-27-085-08W4/0,0854082707BZ0,AMOCO FO-40 GREGOIRE EX AD7-27-85-8,0998,0000000,A001,0000000,Z0000001,RecCertified,19740206,...,0342.90,0002000000,19760522,N/A,ABD,N/A,N/A,-111.17259,56.396349,10Z/07-27-085-08W4/0


In [11]:
m3 = folium.Map(location=[53.5,-114], zoom_start=5)
mc3 = MarkerCluster()

for row in aer.itertuples():
    mc3.add_child(folium.Marker(location=[row.LATITUDE,row.LONGITUDE], popup='https://www.wellwiki.org/wiki/'+row.UWI))
m3.add_child(mc3)
m3
   

KeyboardInterrupt: 

## Conclusion

In this notebook we generated interactive maps of orphan or abandoned wells in Alberta using data from [Orphan Well Association](https://www.orphanwell.ca) and [WellWiki](https://www.wellwiki.org). How are the maps similar and different?

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)